### Install libraries

In [1]:

%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import libraries

In [1]:
from pynput import keyboard, mouse
from ultralytics import YOLO

import cv2
import mss
import numpy as np
import random

#### Choose model

In [2]:
model = YOLO("F://FarmMergeValleyDetector//models//best_yolo11x_17_10_2024.pt")

### Util functions

##### Constant variables

In [3]:
COLOR = (255, 0, 0)
CONFIDENCE_LEVEL = 0.5
OUTPUT_HEIGHT = 720
OUTPUT_WIDTH = 1280
ZOOM_FACTOR = 2

##### Capture screen

In [4]:
def capture_screen(sct):
    screenshot = sct.grab(sct.monitors[1])
    frame = np.array(screenshot)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)  # Convert from BGRA to BGR
    resized_frame = cv2.resize(frame, (OUTPUT_WIDTH, OUTPUT_HEIGHT))
    return resized_frame

##### Count classes in a frame

In [5]:
def count_classes_in_frame(results):
    current_frame_counter = {}
    if results is not None:
        for result in results:
            classes_names = result.names
            for box in result.boxes:
                if box.conf[0] > CONFIDENCE_LEVEL:
                    cls = int(box.cls[0])
                    class_name = classes_names[cls]
                    current_frame_counter[class_name] = current_frame_counter.get(class_name, 0) + 1
    return current_frame_counter

##### Display random class from predicted classes

In [6]:
def select_displayed_class(class_counter, displayed_class):
    if displayed_class and class_counter.get(displayed_class, 0) < 5:
        displayed_class = None
    if displayed_class is None:
        eligible_classes = [class_name for class_name, count in class_counter.items() if count >= 5]
        if eligible_classes:
            displayed_class = random.choice(eligible_classes)
            print(f"Randomly switched to class: {displayed_class}")
    return displayed_class

##### Draw bounding box

In [7]:
def draw_bounding_boxes(frame, results, displayed_class, x1, y1):
    if results is not None and displayed_class is not None:
        for result in results:
            classes_names = result.names
            for box in result.boxes:
                if box.conf[0] > CONFIDENCE_LEVEL:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls = int(box.cls[0])
                    class_name = classes_names[cls]
                    if displayed_class == class_name:
                        cv2.rectangle(frame, (x1, y1), (x2, y2), COLOR, 2)

##### Enhance image

In [19]:
def enhance_image(image, sigma=1.0, alpha=1.5):
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    sharpened = cv2.addWeighted(image, 1 + alpha, blurred, -alpha, 0)
    return sharpened

##### Zoom screen

In [9]:
def zoom_image(image, quadrant):
    height, width = image.shape[:2]
    center_x, center_y = int(width / 2), int(height / 2)

    new_width, new_height = int(width / ZOOM_FACTOR), int(height / ZOOM_FACTOR)

    if quadrant == 1:  # Top-left
        x1, y1 = 0, 0
        x2, y2 = new_width, new_height
    elif quadrant == 2:  # Top-right
        x1, y1 = center_x, 0
        x2, y2 = center_x + new_width, new_height
    elif quadrant == 3:  # Bottom-left
        x1, y1 = 0, center_y
        x2, y2 = new_width, center_y + new_height
    elif quadrant == 4:  # Bottom-right
        x1, y1 = center_x, center_y
        x2, y2 = center_x + new_width, center_y + new_height
    else:
        raise ValueError("Quadrant must be between 1 and 4.")

    x1 = max(x1, 0)
    y1 = max(y1, 0)
    x2 = min(x2, width)
    y2 = min(y2, height)

    cropped_image = image[y1:y2, x1:x2]
    zoomed_image = cv2.resize(cropped_image, (width, height))

    x_scale = width / (x2 - x1)
    y_scale = height / (y2 - y1)

    return zoomed_image, x1, y1, x_scale, y_scale


##### On press triggers the prediction

In [10]:
def on_press(key):
    global trigger_inference
    try:
        if key.char == 'p':
            trigger_inference = True
    except AttributeError:
        pass

In [11]:
def on_click(x, y, button, pressed):
    global trigger_inference
    if pressed and button == mouse.Button.right:
        trigger_inference = True

### Live detection of Farm Merge Valley objects.

In [24]:
sct = mss.mss()
results = None
trigger_inference = False

keyboard_listener = keyboard.Listener(on_press=on_press)
keyboard_listener.start()
mouse_listener = mouse.Listener(on_click=on_click)
mouse_listener.start()

class_counter = {}
displayed_class = None
x1 = 0
y1 = 0
x_scale = 0
y_scale = 0

while True:
    resized_frame = capture_screen(sct)

    if trigger_inference:
        # enhanced_frame = enhance_image(resized_frame)
        # cv2.imshow("Farm Merge Valley Object Detectors", enhanced_frame)
        results = model(resized_frame)
        trigger_inference = False

        current_frame_counter = count_classes_in_frame(results)

        for class_name, count in current_frame_counter.items():
            class_counter[class_name] = class_counter.get(class_name, 0) + count

        displayed_class = select_displayed_class(class_counter, displayed_class)

        class_counter = {}

    draw_bounding_boxes(resized_frame, results, displayed_class, x1, y1)

    cv2.imshow("Farm Merge Valley Object Detector", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()


0: 384x640 6 carrot_1s, 3 carrot_2s, 1 carrot_3, 1 chicken_1, 2 chicken_2s, 3 corn_1s, 4 corn_2s, 1 corn_3, 3 cow_2s, 1 cow_3, 2 goat_1s, 2 goat_2s, 4 goat_3s, 5 pig_1s, 3 pig_2s, 3 pig_3s, 4 sheep_1s, 2 sheep_2s, 4 sheep_3s, 4 soybeans_1s, 2 soybeans_3s, 7 sugarcane_1s, 3 sugarcane_2s, 4 sugarcane_3s, 9 sunflower_1s, 4 sunflower_2s, 2 sunflower_3s, 3 wheat_1s, 3 wheat_2s, 1 wheat_3, 601.7ms
Speed: 6.4ms preprocess, 601.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Randomly switched to class: sugarcane_1

0: 384x640 6 carrot_1s, 3 carrot_2s, 1 carrot_3, 1 chicken_1, 2 chicken_2s, 3 corn_1s, 4 corn_2s, 1 corn_3, 3 cow_2s, 1 cow_3, 2 goat_1s, 2 goat_2s, 4 goat_3s, 5 pig_1s, 3 pig_2s, 3 pig_3s, 5 sheep_1s, 3 sheep_2s, 4 sheep_3s, 3 soybeans_1s, 2 soybeans_3s, 2 sugarcane_1s, 5 sugarcane_2s, 4 sugarcane_3s, 9 sunflower_1s, 4 sunflower_2s, 2 sunflower_3s, 3 wheat_1s, 3 wheat_2s, 1 wheat_3, 504.7ms
Speed: 1.5ms preprocess, 504.7ms inference, 1.0ms postprocess per imag